In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

# Define paths to the dataset
train_dataset_path = '/Users/Sanmesh/Tasks Internship/archive/dogs_vs_cats/train/'
test_dataset_path = '/Users/Sanmesh/Tasks Internship/archive/dogs_vs_cats/test'


# Image dimensions
IMG_SIZE = 128

def extract_hog_features(image):
    """Extract HOG features from an image."""
    features, _ = hog(image, 
                      orientations=9, 
                      pixels_per_cell=(8, 8), 
                      cells_per_block=(2, 2), 
                      block_norm='L2-Hys', 
                      visualize=True, 
                      multichannel=True)
    return features

def load_images_from_folder(folder, limit=500):
    """Load images from folder and extract HOG features."""
    features = []
    labels = []
    count = 0
    for img in os.listdir(folder):
        if count >= limit:
            break
        try:
            label = 'dog' if 'dog' in img else 'cat'  # Assuming filenames contain 'dog' or 'cat'
            img_path = os.path.join(folder, img)
            img_array = cv2.imread(img_path)
            img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            hog_features = extract_hog_features(img_array)
            features.append(hog_features)
            labels.append(label)
            count += 1
        except Exception as e:
            print(f"Error loading image {img}: {e}")
            continue
    return np.array(features), np.array(labels)

# Load a smaller subset of the dataset
X_train, y_train = load_images_from_folder(train_dataset_path, limit=500)  # Limit to 500 images
X_test, y_test = load_images_from_folder(test_dataset_path, limit=500)  # Limit to 500 images

# Check if images were loaded
if len(X_train) == 0 or len(y_train) == 0 or len(X_test) == 0 or len(y_test) == 0:
    print("No images were loaded. Please check the dataset path and structure.")
else:
    # Encode the labels
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Train the SVM model
    svm_model = SVC(kernel='linear')
    svm_model.fit(X_train, y_train_encoded)

    # Predict on the test set
    y_pred = svm_model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_encoded, y_pred)
    print(f'Accuracy: {accuracy * 100:.2f}%')

    def classify_image(filepath):
        """Classify a single image."""
        img_array = cv2.imread(filepath)
        img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        hog_features = extract_hog_features(img_array)
        prediction = svm_model.predict([hog_features])
        return label_encoder.inverse_transform(prediction)[0]

    def upload_image():
        """Upload and classify an image using a GUI."""
        filepath = filedialog.askopenfilename()
        if filepath:
            image = Image.open(filepath)
            image = image.resize((250, 250), Image.ANTIALIAS)
            img = ImageTk.PhotoImage(image)
            panel = tk.Label(root, image=img)
            panel.image = img
            panel.grid(row=1, column=0, padx=10, pady=10)
            
            # Classify the image
            result = classify_image(filepath)
            result_label.config(text=f'Prediction: {result}')

    root = tk.Tk()
    root.title("Cat and Dog Classifier")

    upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
    upload_btn.grid(row=0, column=0, padx=10, pady=10)

    result_label = tk.Label(root, text="Prediction: ")
    result_label.grid(row=2, column=0, padx=10, pady=10)

    root.mainloop()

Error loading image cats: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading image dogs: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading image cats: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading image dogs: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

No images were loaded. Please check the dataset path and structure.
